# DSCI-560 Lab 4: Real-Time Algorithmic Trading


## Real-Time Data Collection


In [4]:
%%writefile data_stream.py
import yfinance as yf
from datetime import datetime

def stream_price(symbol="AAPL"):
    ticker = yf.Ticker(symbol)
    data = ticker.history(period="1d", interval="1m")
    if data.empty:
        return None
    return {
        "time": datetime.now(),
        "symbol": symbol,
        "price": float(data["Close"].iloc[-1]),
        "volume": int(data["Volume"].iloc[-1])
    }


Overwriting data_stream.py


## Mock Trading Environment


In [20]:
%%writefile portfolio.py
class Portfolio:
    def __init__(self, initial_cash=100000):
        self.cash = initial_cash
        self.positions = {}
        self.history = []

    def buy(self, symbol, price, shares):
        cost = price * shares
        if self.cash >= cost:
            self.cash -= cost
            self.positions[symbol] = self.positions.get(symbol, 0) + shares

    def sell(self, symbol, price, shares):
        if symbol not in self.positions:
            return
        if self.positions[symbol] < shares:
            return
        self.cash += price * shares
        self.positions[symbol] -= shares
        if self.positions[symbol] == 0:
            del self.positions[symbol]

    def value(self, prices):
        total = self.cash
        for s, sh in self.positions.items():
            total += prices.get(s, 0) * sh
        return total


Overwriting portfolio.py


## Technical Indicators


In [6]:
%%writefile indicators.py
def sma(series, window=10):
    return series.rolling(window).mean()

def rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))


Writing indicators.py


## Trading Strategy


In [7]:
%%writefile strategies.py
def hybrid_strategy(df):
    if len(df) < 15:
        return 0
    price = df["price"].iloc[-1]
    sma_val = df["sma"].iloc[-1]
    rsi_val = df["rsi"].iloc[-1]
    if price > sma_val and rsi_val < 70:
        return 1
    if price < sma_val or rsi_val > 70:
        return -1
    return 0


Writing strategies.py


## Performance Evaluation


In [8]:
%%writefile metrics.py
import numpy as np
import pandas as pd

def calculate_metrics(values):
    returns = pd.Series(values).pct_change().dropna()
    total_return = (values[-1] / values[0]) - 1
    sharpe = 0
    if returns.std() != 0:
        sharpe = (returns.mean() / returns.std()) * np.sqrt(252)
    return total_return, sharpe


Writing metrics.py


## Main Execution


In [76]:
%%writefile main.py
import pandas as pd
import yfinance as yf
from portfolio import Portfolio
from indicators import sma, rsi
from strategies import hybrid_strategy
from metrics import calculate_metrics

SYMBOLS = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA",
    "META", "TSLA", "AMD", "SPY", "QQQ"
]

ALLOCATION = 0.30

portfolio = Portfolio(1000000)
dfs = {}

for symbol in SYMBOLS:
    data = yf.download(symbol, period="1d", interval="1m", auto_adjust=False, progress=False)
    if data.empty:
        continue

    df = pd.DataFrame()
    df["time"] = data.index
    df["price"] = data["Close"].values
    df["volume"] = data["Volume"].values
    df.reset_index(drop=True, inplace=True)

    df["sma"] = sma(df["price"], 5)
    df["rsi"] = rsi(df["price"], 7)

    dfs[symbol] = df

min_len = min(len(df) for df in dfs.values())

for i in range(15, min_len):
    prices = {}

    for symbol, df in dfs.items():
        current_price = df.loc[i, "price"]
        prices[symbol] = current_price

        signal = hybrid_strategy(df.iloc[: i + 1])

        if signal == 1:
            shares = int((portfolio.cash * ALLOCATION) // current_price)
            if shares > 0:
                portfolio.buy(symbol, current_price, shares)

        elif signal == -1:
            portfolio.sell(symbol, current_price, portfolio.positions.get(symbol, 0))

        print(
            f"{df.loc[i,'time']} | {symbol} | "
            f"Signal: {signal} | "
            f"Cash: ${portfolio.cash:,.2f} | "
            f"Positions: {portfolio.positions} | "
            f"Total Value: ${portfolio.value(prices):,.2f}"
        )

    portfolio.history.append(portfolio.value(prices))

ret, sharpe = calculate_metrics(portfolio.history)

print("\nFinal Results")
print("Total Return:", round(ret * 100, 2), "%")
print("Sharpe Ratio:", round(sharpe, 2))


Overwriting main.py


In [77]:
%%writefile requirements.txt
yfinance
pandas
numpy
matplotlib

Overwriting requirements.txt


## Run the Trading Simulation


In [78]:
!pip install -r requirements.txt
!python main.py

2026-02-06 14:47:00+00:00 | AAPL | Signal: -1 | Cash: $1,000,000.00 | Positions: {} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | MSFT | Signal: 1 | Cash: $700,133.20 | Positions: {'MSFT': 751} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | GOOGL | Signal: 1 | Cash: $490,263.41 | Positions: {'MSFT': 751, 'GOOGL': 642} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | AMZN | Signal: -1 | Cash: $490,263.41 | Positions: {'MSFT': 751, 'GOOGL': 642} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | NVDA | Signal: -1 | Cash: $490,263.41 | Positions: {'MSFT': 751, 'GOOGL': 642} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | META | Signal: 1 | Cash: $343,838.75 | Positions: {'MSFT': 751, 'GOOGL': 642, 'META': 221} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | TSLA | Signal: 1 | Cash: $240,953.77 | Positions: {'MSFT': 751, 'GOOGL': 642, 'META': 221, 'TSLA': 253} | Total Value: $1,000,000.00
2026-02-06 14:47:00+00:00 | AMD | Signal: 1 | Cas

## Results Summary
